## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [3]:
# Create y_df with the Attrition and Department columns
y_df['Attrition'] = y_df['Attrition'].map({'Yes': 1, 'No': 0})


NameError: name 'y_df' is not defined

In [ ]:
# Create a list of at least 10 column names to use as X data
list_column = ['HourlyRate',
               'PerformanceRating',
               'TotalWorkingYears',
               'YearsAtCompany',
               'YearsInCurrentRole',
               'YearsSinceLastPromotion',
               'DistanceFromHome',
               'WorkLifeBalance',
               'JobSatisfaction',
               'EnvironmentSatisfaction']

# Create X_df using your selected columns
X_df = attrition_df[list_column]

# Show the data types for X_df
X_df.dtypes

In [ ]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state = 32)
y_attrition_train, y_attrition_test = train_test_split(y_df['Attrition'], test_size=0.2, random_state=32)
y_department_train, y_department_test = train_test_split(y_department, test_size=0.2, random_state=32)

In [ ]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
# Data types are already numeric.


In [ ]:
# Create a StandardScaler
X_scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder for the Department column
department_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop=None)

# Fit the encoder to the training data
department_encoder.fit(np.array(y_train['Department']).reshape(-1, 1))

# Create two new variables by applying the encoder to the training and testing data
y_department = pd.get_dummies(y_df['Department'])
y_department_train = department_encoder.transform(np.array(y_train['Department']).reshape(-1, 1))
y_department_test = department_encoder.transform(np.array(y_test['Department']).reshape(-1, 1))

# Display the transformed training data
print(y_department_train)

In [ ]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop=None)

# Fit the encoder to the training data
attrition_encoder.fit(np.array(y_train['Attrition']).reshape(-1, 1))

# Create two new variables by applying the encoder
# to the training and testing data
y_attrition_train = attrition_encoder.transform(np.array(y_train['Attrition']).reshape(-1, 1))
y_attrition_test = attrition_encoder.transform(np.array(y_test['Attrition']).reshape(-1, 1))

# Display the transformed training data
print(y_attrition_train)

## Part 2: Create, Compile, and Train the Model

In [ ]:
# Find the number of columns in the X training data.
num_columns = X_df.shape[1]
print("Number of columns in X training data:", num_columns)

# Create the input layer
input_layer = layers.Input(shape=(X_df.shape[1],), name='input_features')


# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation='relu')(input_layer)
shared_layer2 = layers.Dense(32, activation='relu')(shared_layer1)

In [ ]:
# Create a branch for Department with a hidden layer and an output layer
# Create the hidden layer
department_hidden = layers.Dense(32, activation='relu', name='department_hidden')(shared_layer2)

# Create the output layer
department_output = layers.Dense(3, activation='softmax', name='department_output')(department_hidden)

In [ ]:
# Create a branch for Attrition with a hidden layer and an output layer
# Create the hidden layer
attrition_hidden = layers.Dense(32, activation='relu', name='attrition_hidden')(shared_layer2)

# Create the output layer
attrition_output = layers.Dense(1, activation='sigmoid', name='attrition_output')(attrition_hidden)

In [ ]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer="adam",
              loss={"department": "categorical_crossentropy", "attrition": "binary_crossentropy"},
              metrics={"department": "accuracy", "attrition": "accuracy"})

# Summarize the model
model.summary()

In [ ]:
# Convert scaled training data type to integer
X_train_scaled = X_train_scaled.astype(int)

In [ ]:
# Train the model
fit_model = model.fit(X_train_scaled, [y_department_train, y_attrition_train], epochs=50)


In [ ]:
# Check the data types
print(X_test.dtypes)
print(y_test.dtypes)


In [ ]:
# Convert to numeric values
X_test = X_test.apply(pd.to_numeric, errors='coerce')
y_test = y_test.apply(pd.to_numeric, errors='coerce')


In [ ]:
# Drop null values
X_test = X_test.fillna(0)  # Replace NaN with 0
y_test = y_test.fillna(0)


In [ ]:
# Convert to numpy arrays
X_test = np.array(X_test)
y_test = np.array(y_test)


In [ ]:
y_test = (y_test['attrition'], y_test['department'])  # Adjust column names as needed


In [ ]:
# Evaluate the model with the testing data
model.evaluate(X_test, y_test)



In [ ]:
# Print the accuracy for both department and attrition


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1.
2.
3.